In [1]:
!nvidia-smi

Thu Jun 16 16:55:03 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.156.00   Driver Version: 450.156.00   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM3...  On   | 00000000:39:00.0 Off |                    0 |
| N/A   43C    P0    56W / 350W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install df_engine

In [3]:
!pip install fasttext

In [4]:
!pip install pydantic==1.8.2

In [5]:
!pip install transformers==4.10.3

In [6]:
!pip install googletrans==3.1.0a0

In [7]:
import transformers
transformers.__version__

2022-06-16 16:55:18.197972: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


'4.10.3'

In [8]:
import pandas as pd
import transformers
from transformers import BertModel, BertTokenizer, BertConfig, GPT2LMHeadModel, GPT2Tokenizer
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

import random

import numpy as np
from tqdm import tqdm
import sys
import os
import pickle
import random
from dataclasses import dataclass, field
from typing import List, Dict, Union
import json
from tqdm import tqdm
from sklearn.metrics import *

import logging
import re
from typing import Optional, Union, Any
from df_engine.core.keywords import TRANSITIONS, RESPONSE
from df_engine.core import Context, Actor
import df_engine.conditions as cnd
import df_engine.labels as lbl
from df_engine.core.types import NodeLabel3Type
import os
import requests
from nltk.tokenize import sent_tokenize

import warnings
warnings.simplefilter("ignore")

In [9]:
#intentcls starts

In [10]:
config = BertConfig.from_pretrained("sentence-transformers/LaBSE", num_labels=86)
tokenizer = BertTokenizer.from_pretrained("sentence-transformers/LaBSE")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [11]:
#load
model_state_dict = torch.load("best_model_state.bin")

In [12]:
df = pd.read_csv("df_train_translated_new.csv", encoding="latin1").fillna(method="ffill")
class_names = {}

for w, t in zip(df["intents_nums"], df["intent"]):
      class_names[w] = t

In [13]:
class SentimentClassifier(nn.Module):

    def __init__(self, n_classes):
        super(SentimentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained("sentence-transformers/LaBSE", state_dict=model_state_dict)
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
  
    def forward(self, input_ids, attention_mask, return_dict):
        _, pooled_output = self.bert(
        input_ids=input_ids,
        attention_mask=attention_mask,
        return_dict = False
    )
        output = self.drop(pooled_output)
        return self.out(output)

In [14]:
model_loaded = SentimentClassifier(len(class_names))
model_loaded.load_state_dict(model_state_dict)
model_loaded.to(device)
model_loaded = model_loaded.eval()

Some weights of the model checkpoint at sentence-transformers/LaBSE were not used when initializing BertModel: ['out.weight', 'out.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
def get_intents(model,text):
    encoded_text = tokenizer.encode_plus(
    text,
    max_length=160,
    add_special_tokens=True,
    return_token_type_ids=False,
    padding = 'max_length',
    return_attention_mask=True,
    return_tensors='pt',
    )
    input_ids = encoded_text['input_ids'].to(device)
    attention_mask = encoded_text['attention_mask'].to(device)

    output = model(input_ids, attention_mask, return_dict=False)
    _, preds = torch.max(output, dim=1)
    return class_names[int(preds)]

In [16]:
print(get_intents(model_loaded, 'and its address'))

request_addr


In [17]:
#ner starts

In [18]:
tokenizer_gpt = GPT2Tokenizer.from_pretrained("sberbank-ai/mGPT")
model = GPT2LMHeadModel.from_pretrained("sberbank-ai/mGPT")

In [19]:
model=model.to(device)

In [20]:
train_lang = 'en'
prompts = {0: "<s>lang: ", 1: "\nTagged sentence: ", 2: '</s>'}
def format_train_data(sent):
    sent = sent.replace('\n','@@@')
    sent = re.sub(r'\s+', ' ', sent)
    sent = sent.replace(' ','_')
    sent = ' '.join(random.choices(re.sub('@@@', ' ', sent).strip().split(), k=45))
    result = prompts[0] +train_lang+ prompts[1] +sent+ prompts[2]
    return result

def format_test_data(sent, i, lang):
    sent = sent.replace('\n','@@@')
    sent = re.sub(r'\s+', ' ', sent)
    sent = sent.replace(' ','_')
    sent = re.sub('@@@', ' ', sent).strip().split()
    result = prompts[0] +lang+ prompts[1] +' '.join(sent[:i]) + ' ' \
             + sent[i].split('_')[0] + '_'
    return ' '.join(sent[:i]) + ' ' + sent[i].split('_')[0] + '_'

In [21]:
tagset = {line.strip().split()[-1] for line in open("en_data/train.txt") if line.strip()}

In [39]:
en_data = format_test_data(open("en_data/test.txt").read(), 42, 'en')
de_data = format_test_data(open("de_data/valid.txt").read(), 42, 'de')
ru_data = format_test_data(open("ru_data/train.txt").read(), 42, 'ru')

In [23]:
def hypotheses(sent):
    pref = en_data+de_data+ru_data+sent
    hyps = [pref+tag for tag in tagset]
    return hyps, tagset

def choose_tag(sentence):
    losses = {}
    hyps, tagset = hypotheses(sentence)
    for tag, hyp in zip(tagset, hyps):
        input_ids = tokenizer_gpt.encode(hyp, return_tensors='pt').cuda(device)
        model_output = model.forward(input_ids, labels=input_ids)
        losses[tag] = float(model_output["loss"].cpu().detach().numpy())
    return min(losses, key=losses.get)

def get_ner_tags(text_to_tag):
    text_tokens = text_to_tag.split()
    sentence = ''
    text_tags = []
    result = {}
    for token in text_tokens:
        sentence += token+'_'
        next_tag = choose_tag(sentence)
        text_tags.append(next_tag)
        sentence += next_tag+ ' '
    for word, tag in zip(text_to_tag.split(), text_tags):
        result[word] = tag
    return result

In [24]:
get_ner_tags('japanese restaurant')

{'japanese': 'B-food', 'restaurant': 'O'}

In [25]:
#language detection and translation starts

In [26]:
texts = ['nördAichen und Stadt orientalisch Restaurant', 'Ажылдаар ужурлуг мен', 'Китайский ресторан', 'cuisine japonaise']

In [27]:
import fasttext
fst = fasttext.load_model('lid.176.bin')
for text in texts:
    print(fst.predict(text, k=1)[0][0].replace('__label__',''))

de
ky
ru
fr


In [28]:
from googletrans import Translator, constants
translator = Translator()
translation = translator.translate('What part of town do you have in mind?', dest="ru")
print(translation.text)

Какую часть города вы имеете в виду?


In [29]:
#database processing and connecting starts

In [30]:
with open('akadem.txt') as f:
    db_results = f.read()
import ast
db_results = db_results.replace('\u200b', '')
database = ast.literal_eval(db_results)

In [31]:
print(database[0])

{'name': 'Гуси ресторан-пивоварня', 'food': 'европейская', 'pricerange': '$$ - $$$', 'area': 'нижняя зона', 'addr': 'Николаева, 12/2, 1 этаж, Академгородок м-н, Советский район, Новосибирск', 'phone': '+7 (383) 310-16-03', 'postcode': '630090'}


In [32]:
food = 'ирландская'
area = 'верхняя зона'
pricerange = '$$ - $$$'
result = []
for i in range(len(database)):
    if ('food', food) in database[i].items() and ('area', area) in database[i].items() and ('pricerange', pricerange) in database[i].items():
        result.append(database[i]['name'])
               
print(result)      

['Клевер Айриш Паб']


In [33]:
#bot starts

In [49]:
logger = logging.getLogger(__name__)


def unknown_slots(ctx: Context):
    if "slots" not in ctx.misc.keys() or not ctx.misc['slots']:
         ctx.misc["slots"] = {
             "food": None,
             "pricerange": None,
             "area": None,
             "phone": None,
             "addr": None,
             "postcode": None,
             "name": None}
    ctx.misc["unknown_slots"] = []
    for key, value in ctx.misc["slots"].items():
        if value is None:
            ctx.misc['unknown_slots'].append(key)
    print("unknown slots", ctx.misc["slots"])
    return ctx


def get_some_intent(ctx: Context, actor: Actor):
    request = ctx.last_request
    intent = get_intents(model_loaded, request)
    detected_language = fst.predict(request, k=1)[0][0].replace('__label__','')
    ctx.misc["intent_detection"] = {request: intent}
    ctx.misc["language"] = {request: detected_language}
    return ctx


def get_ner_tag(ctx: Context, actor: Actor):
    unknown_slots(ctx)
    request = ctx.last_request
    request_slots = get_ner_tags(request)
    print('GET NER TAG', request_slots)
    for key, value in request_slots.items():
        if value == 'O':
            pass
        elif value.split('-')[1].endswith("pricerange"):
            ctx.misc["slots"]["pricerange"] = key
        elif value.split('-')[1].endswith("area"):
            ctx.misc["slots"]["area"] = key
        elif value.split('-')[1].endswith("food"):
            ctx.misc["slots"]["food"] = key
    unknown_slots(ctx)
    return ctx


def turn_handler(
    in_request: str, ctx: Union[Context, str, dict], actor: Actor, true_out_response: Optional[str] = None
):
    ctx = Context.cast(ctx)
    ctx.add_request(in_request)

    ctx.misc['unknown_slots'] = []
    ctx.misc["intent_detection"] = {}
    ctx.misc["language"] = {} 
    request = ctx.last_request
    get_some_intent(ctx, actor)
    get_ner_tag(ctx, actor)
    
    ctx = actor(ctx)
    out_response = ctx.last_response
    print(ctx.misc)
    if true_out_response is not None and true_out_response != out_response:
        msg = f"in_request={in_request} -> true_out_response != out_response: {true_out_response} != {out_response}"
        raise Exception(msg)
    else:
        logging.info(f"in_request={in_request} -> {out_response}")
    return out_response, ctx


def get_item_from_db(ctx: Context):
    pricerange = ''
    area = ''
    food = ''
    if ctx.misc['slots']['pricerange'] == 'дешёвый' or ctx.misc['slots']['pricerange'] == 'cheap':
        pricerange = '$'
    elif ctx.misc['slots']['pricerange'] == 'средний' or ctx.misc['slots']['pricerange'] == 'moderate':
        pricerange = '$$'
    elif ctx.misc['slots']['pricerange'] == 'дорогой' or ctx.misc['slots']['pricerange'] == 'expensive':
        pricerange = '$$ - $$$'
    
    
    if ctx.misc['slots']['food'] == 'итальянский' or ctx.misc['slots']['food'] == 'italian' or ctx.misc['slots']['food'] == 'italien':
        food = 'итальянская'
    elif ctx.misc['slots']['food'] == 'японский' or ctx.misc['slots']['food'] == 'japanese' or ctx.misc['slots']['food'] == 'japonais':
        food = 'японская'
    elif ctx.misc['slots']['food'] == 'европейский' or ctx.misc['slots']['food'] == 'european' or ctx.misc['slots']['food'] == 'europeen':
        food = 'европейская'
    elif ctx.misc['slots']['food'] == 'американский' or ctx.misc['slots']['food'] == 'american' or ctx.misc['slots']['food'] == 'americain':
        food = 'американская'
    elif ctx.misc['slots']['food'] == 'ирландский' or ctx.misc['slots']['food'] == 'irish' or ctx.misc['slots']['food'] == 'irlandais':
        food = 'ирландская'
        
    if ctx.misc['slots']['area'] == 'west' or ctx.misc['slots']['area'] == "l'ouest" or ctx.misc['slots']['area'] == 'запад':
        area = 'верхняя зона'
    elif ctx.misc['slots']['area'] == 'south' or ctx.misc['slots']['area'] == "sud" or ctx.misc['slots']['area'] == 'юг':
        area = 'верхняя зона'
    elif ctx.misc['slots']['area'] == 'north' or ctx.misc['slots']['area'] == "nord" or ctx.misc['slots']['area'] == 'север':
        area = 'нижняя зона'    
    elif ctx.misc['slots']['area'] == 'east' or ctx.misc['slots']['area'] == "l'est" or ctx.misc['slots']['area'] == 'восток':
        area = 'нижняя зона'
    
    result = []
    for i in range(len(database)):
        if ('food', food) in database[i].items() and ('area', area) in database[i].items() and ('pricerange', pricerange) in database[i].items():
            result.append(database[i]['name'])
    num_results = len(result)
    if num_results != 0:
        response = f"I've found {num_results} results. {''.join(result)} is a great restaurant for you."
    else:
        response = f"Unfortunately I've found {num_results} results."
    return response

def central_response(ctx: Context, actor: Actor, *args, **kwargs) -> Any:
    request = ctx.last_request
    if "slots" not in ctx.misc.keys():
         ctx.misc["slots"] = {
             "food": None,
             "pricerange": None,
             "area": None,
             "phone": None,
             "addr": None,
             "postcode": None,
             "name": None}
    if "unknown_slots" not in ctx.misc.keys():
         ctx.misc["unknown_slots"] = []
    if "language" not in ctx.misc.keys():
        ctx.misc["language"] = {}
    unkn_slots = ctx.misc["unknown_slots"]
    if request == 'text':
        pass
    else:
        necessary = ['food', 'pricerange', 'area']
        language = ctx.misc["language"][request]
        if not any(x in necessary for x in unkn_slots):
            response = get_item_from_db(ctx)
            if language == 'en':
                return response
            else:
                translation = translator.translate(response, dest=language)
                return translation.text
        else:
            necessary = ['food', 'pricerange', 'area']
            for v in necessary:
                if v in unkn_slots:
                    response = f"What {v} do you have in mind?"
                    if language == 'en':
                        return response
                    else:
                        translation = translator.translate(response, dest=language)
                        return translation.text 
                else:
                    continue


def custom_condition(ctx: Context, actor: Actor, *args, **kwargs) -> bool:
    try:
        necessary = ['food', 'pricerange', 'area']
        if any(x in necessary for x in ctx.misc['unknown_slots']):
            return True
        else:
            return False
    except KeyError:
        return True

def not_custom_condition(ctx: Context, actor: Actor, *args, **kwargs) -> bool:
    return not custom_condition(ctx, actor, *args, **kwargs)

def central_condition(ctx: Context, actor: Actor, *args, **kwargs) -> bool:
    try:
        request = ctx.last_request
        if 'inform' in ctx.misc["intent_detection"][request]:
            return True
        else:
            return False
    except KeyError:
        return True   


def hi_condition(ctx: Context, actor: Actor, *args, **kwargs) -> bool:
    try:
        request = ctx.last_request
        if ctx.misc["intent_detection"][request] == 'hello':
            return True
        else:
            return False
    except KeyError:
        return True
    
def hi_response(ctx: Context, actor: Actor, *args, **kwargs) -> Any:
    request = ctx.last_request
    if "language" not in ctx.misc.keys():
        ctx.misc["language"] = {}
    if request == 'text':
        pass
    else:        
        language = ctx.misc["language"][request]
        response = "Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?"
        if language == 'en':
             return response
        else:
            translation = translator.translate(response, dest=language)
            return translation.text 
    
    
def bye_condition(ctx: Context, actor: Actor, *args, **kwargs) -> bool:
    try:
        request = ctx.last_request
        if 'bye' in ctx.misc["intent_detection"][request]:
            return True
        else:
            return False
    except KeyError:
        return True
    
def bye_response(ctx: Context, actor: Actor, *args, **kwargs) -> Any:
    request = ctx.last_request
    if "language" not in ctx.misc.keys():
        ctx.misc["language"] = {}
    if request == 'text':
        pass
    else:
        language = ctx.misc["language"][request]
        response = 'Thank you for using Cambridge restaurant system! Goodbye!'
        if language == 'en':
             return response
        else:
            translation = translator.translate(response, dest=language)
            return translation.text 

    
def request_condition(ctx: Context, actor: Actor, *args, **kwargs) -> bool:
    try:
        request = ctx.last_request
        if 'request' in ctx.misc["intent_detection"][request]:
            return True
        else:
            return False
    except KeyError:
        return True
    
def request_response(ctx: Context, actor: Actor, *args, **kwargs) -> Any:
    request = ctx.last_request
    if "language" not in ctx.misc.keys():
        ctx.misc["language"] = {}
    if request == 'text':
        pass
    else:
        language = ctx.misc["language"][request]
        response = "Sorry, I'm still developing, now I don't work with requests"
        if language == 'en':
             return response
        else:
            translation = translator.translate(response, dest=language)
            return translation.text

    
def fallback_response(ctx: Context, actor: Actor, *args, **kwargs) -> Any:
    request = ctx.last_request
    if "language" not in ctx.misc.keys():
        ctx.misc["language"] = {}
    if request == 'text':
        pass
    else:
        language = ctx.misc["language"][request]
        response = "Sorry, I don't understand. Could you make it more clear?"
        if language == 'en':
             return response
        else:
            translation = translator.translate(response, dest=language)
            return translation.text    
        
        
def fallback_condition(ctx: Context, actor: Actor, *args, **kwargs) -> bool:
    try:
        request = ctx.last_request
        if 'unknown' in ctx.misc["intent_detection"][request]:
            return True
        else:
            return False
    except KeyError:
        return True  


def print_misc(ctx: Context, actor: Actor):
    print(f"{ctx.misc}")
    return ctx
    
    
script = {
    "global_flow": {
        "start_node": {  
            RESPONSE: "",
            TRANSITIONS: {
                ("global_flow", "greeting_node"): hi_condition, 
                ("global_flow", "central_node"): central_condition,
                ("global_flow", "request_node"): request_condition,
                ("global_flow", "bye_node"): bye_condition,
                lbl.to_fallback(): fallback_condition,
            },
        },
        "fallback_node": {  # We get to this node if an error occurred while the agent was running
            RESPONSE: fallback_response,
            TRANSITIONS: {
                ("global_flow", "greeting_node"): hi_condition,
                ("global_flow", "request_node"): request_condition,
                ("global_flow", "central_node"): central_condition,
                ("global_flow", "bye_node"): bye_condition,
            },
        },
        "greeting_node": {
            RESPONSE: hi_response,
            TRANSITIONS: {
                lbl.repeat(): hi_condition,
                ("global_flow", "central_node"): central_condition,
                ("global_flow", "request_node"): request_condition,
                lbl.to_fallback(): fallback_condition,
                ("global_flow", "bye_node"): bye_condition,
            },
        },
        "central_node": {
            RESPONSE: central_response, 
            TRANSITIONS: {
                ("global_flow", "greeting_node"): hi_condition,
                ("global_flow","bye_node"): bye_condition,
                ("global_flow", "request_node"): request_condition,
                lbl.repeat(): not_custom_condition,
                lbl.repeat(): custom_condition,
            },
        },
        "bye_node": {
            RESPONSE: bye_response,
            TRANSITIONS: {
                lbl.repeat(): bye_condition,
                ("global_flow", "greeting_node"): hi_condition,
                ("global_flow", "central_node"): central_condition,
                ("global_flow", "request_node"): request_condition,
                lbl.to_fallback(): fallback_condition, 
            },
        },
        "request_node": {
            RESPONSE: request_response,
            TRANSITIONS: {
                lbl.repeat(): request_condition,
                ("global_flow", "greeting_node"): hi_condition,
                ("global_flow", "central_node"): central_condition,
                lbl.to_fallback(): fallback_condition,
            },
        },
    },
}

actor = Actor(
    script,
    start_label=("global_flow", "start_node"),
    fallback_label=("global_flow", "fallback_node"),
    label_priority=1.0,
)


# testing
testing_dialog = [
    ("hi", "Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?"),
    ("i'm fine, how are you?", "Good. What do you want to talk about?"),
    ("i want cheap restaurant.", "What kind of food would you like?"),
    ("chinese", "Would you like something in the cheap, moderate, or expensive price range?"),
    ("cheap", "What part of town do you have in mind?"),
    ("west", "API REQUEST."),
    ("next", "Good. What do you want to talk about?"),
    ("previous", "That's all what I know"),
    ("next time", "bye"),
    ("stop", "Ooops"),
    ("previous", "bye"),
    ("stop", "Ooops"),
    ("nope", "Ooops"),
    ("hi", "Hi, how are you?"),
    ("stop", "Ooops"),
    ("Ok, goodbye.", "bye"),
]


def run_test():
    ctx = {}
    for in_request, true_out_response in testing_dialog:
        _, ctx = turn_handler(in_request, ctx, actor, true_out_response=true_out_response)


# interactive mode
def run_interactive_mode(actor):
    ctx = {}
    while True:
        in_request = input("type your answer: ")
        _, ctx = turn_handler(in_request, ctx, actor)

        
if __name__ == "__main__":
    logging.basicConfig(
        format="%(asctime)s-%(name)15s:%(lineno)3s:%(funcName)20s():%(levelname)s - %(message)s", level=logging.INFO
    )
    #run_test()
    run_interactive_mode(actor)

KeyboardInterrupt: Interrupted by user

In [ ]:
#something for telegram

In [ ]:
!pip install df-telegram-connector

In [37]:
from df_telegram_connector.connector import TelegramConnector
from df_telegram_connector.request_provider import PollingRequestProvider

from df_runner import ScriptRunner

In [ ]:
bot = TelegramConnector('5315229065:AAFfKjvkYTpYLLAGpBQa7goxJQrQwvFOb0s')

provider = PollingRequestProvider(bot=bot)

runner = ScriptRunner(
    script=script,
    start_label=("global_flow", "start_node"),
    fallback_label=("global_flow", "fallback_node"),
    db=dict(),
    request_provider=provider,
    pre_annotators=[get_some_intent, get_ner_tag],
    post_annotators=[print_misc],
)


if __name__ == "__main__":
    runner.start()

unknown slots {'food': None, 'pricerange': None, 'area': None, 'phone': None, 'addr': None, 'postcode': None, 'name': None}
GET NER TAG {'привет': 'O'}
unknown slots {'food': None, 'pricerange': None, 'area': None, 'phone': None, 'addr': None, 'postcode': None, 'name': None}
{'intent_detection': {'привет': 'hello'}, 'language': {'привет': 'ru'}, 'slots': {'food': None, 'pricerange': None, 'area': None, 'phone': None, 'addr': None, 'postcode': None, 'name': None}, 'unknown_slots': ['food', 'pricerange', 'area', 'phone', 'addr', 'postcode', 'name']}
unknown slots {'food': None, 'pricerange': None, 'area': None, 'phone': None, 'addr': None, 'postcode': None, 'name': None}
GET NER TAG {'expensive': 'B-pricerange', 'restaurant': 'O'}
unknown slots {'food': None, 'pricerange': 'expensive', 'area': None, 'phone': None, 'addr': None, 'postcode': None, 'name': None}
{'intent_detection': {'expensive restaurant': 'inform_pricerange'}, 'language': {'expensive restaurant': 'en'}, 'slots': {'food': 